In [ ]:
# Test KFP Integration

- create an experiment
- create a run
- check run passes

This Notebook is based on the condition Kubeflow pipelines [example](https://github.com/kubeflow/pipelines/blob/master/samples/core/condition/condition_v2.py).

In [ ]:
# Please check the requirements.in file for more details
!pip install -r requirements.txt

In [ ]:
import kfp
import os

from kfp import dsl
from tenacity import retry, stop_after_attempt, wait_exponential

In [ ]:
client = kfp.Client()

In [ ]:
EXPERIMENT_NAME = 'Check GPU' 

In [ ]:
@dsl.component(base_image="kubeflownotebookswg/jupyter-tensorflow-cuda-full:v1.9.0")
def check_gpu() -> str:
    """Check GPU"""
    import tensorflow as tf
    gpu_list = tf.config.list_physical_devices('GPU')
    print("GPU list:", gpu_list)
    return str(len(gpu_list)>0)

def configure_check_gpu(obj):
    """Adds the proxy env vars to the PipelineTask object."""
    return ( obj.add_node_selector_constraint(accelerator = "nvidia.com/gpu").set_accelerator_limit(limit = 1) )

In [ ]:
@dsl.pipeline
def check_gpu_pipeline() -> str:
    """My ML pipeline."""
    check_gpu1 = check_gpu()
    configure_check_gpu(check_gpu1)
    return check_gpu1.output

In [ ]:
# Setting enable_caching to False to overcome https://github.com/canonical/bundle-kubeflow/issues/1067
run = client.create_run_from_pipeline_func(
    check_gpu_pipeline,
    experiment_name=EXPERIMENT_NAME,
    enable_caching=False,
)

In [ ]:
client.list_experiments().experiments

In [ ]:
client.get_run(run.run_id).state

In [ ]:
@retry(
    wait=wait_exponential(multiplier=2, min=1, max=10),
    stop=stop_after_attempt(30),
    reraise=True,
)
def assert_run_succeeded(client, run_id):
    """Wait for the run to complete successfully."""
    status = client.get_run(run_id).state
    assert status == "SUCCEEDED", f"KFP run in {status} state."

In [ ]:
# fetch KFP experiment to ensure it exists
client.get_experiment(experiment_name=EXPERIMENT_NAME)

assert_run_succeeded(client, run.run_id)